**Mount Drive & Install Packages**

First the directory containing the data must be mounted to the Colab virtual machine

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Import relevant libraries for use

In [ ]:
import tensorflow as tf
from tensorflow import keras

tf.__version__
keras.__version__

Paths for training data and test data

In [ ]:
training_data_path = '/content/drive/My Drive/AD Project/Alzheimers-Data-Preprocessed-N4/train/'
valid_data_path = '/content/drive/My Drive/AD Project/Alzheimers-Data-Preprocessed-N4/test/'

Produce trainin set generators to flow images thorugh from preprocessed directory

In [ ]:
from numpy.core.numeric import True_
trainGenerator = tf.keras.preprocessing.image.ImageDataGenerator()

validGenerator = tf.keras.preprocessing.image.ImageDataGenerator()

flowTrainGenerator = trainGenerator.flow_from_directory(training_data_path, target_size=(224,224), color_mode='rgb',shuffle=True,interpolation="nearest",batch_size=64, class_mode='categorical')

flowValidGenerator = validGenerator.flow_from_directory(valid_data_path, target_size=(224,224), color_mode='rgb',shuffle=True,interpolation="nearest",batch_size=64, class_mode='categorical')

**Initialise Metrics**

List of metrics ready for Keras


In [ ]:
METRICS = [
           tf.keras.metrics.CategoricalAccuracy(name="categorical_accuracy", dtype=None),
           tf.keras.metrics.Precision(thresholds=None, top_k=None, class_id=None, name=None, dtype=None),
           tf.keras.metrics.Recall(thresholds=None, top_k=None, class_id=None, name=None, dtype=None),
           tf.keras.metrics.TruePositives(thresholds=None, name=None, dtype=None),
           tf.keras.metrics.TrueNegatives(thresholds=None, name=None, dtype=None),
           tf.keras.metrics.FalsePositives(thresholds=None, name=None, dtype=None),
           tf.keras.metrics.FalseNegatives(thresholds=None, name=None, dtype=None)          
]

**Implementation of VGG16**

My own implementation of VGG16 in Keras with my own adaptations

In [ ]:
#Use sequential strucutre to formulate VG16 CNN architecture

vgmodel = tf.keras.Sequential()

#Initialise input shape for model

inputShape = (224,224,3)

#Define layers of VG16

#Initial convolutional section accpeting 224x224 image - consisting of 64 feature map, 3x3 convolutions X2

vgmodel.add(tf.keras.layers.Conv2D(64,(3,3), input_shape=inputShape, strides = (1,1), activation='relu'))

vgmodel.add(tf.keras.layers.Conv2D(64,(3,3), strides = (1), activation='relu'))

#First maxpooling layer

vgmodel.add(tf.keras.layers.MaxPool2D(pool_size = (3), strides = (2,2)))

#Second convolutional section - consisting of 128 feature map, 3x3 convolutions X2

vgmodel.add(tf.keras.layers.Conv2D(128,(3,3), strides = (1), activation='relu'))

vgmodel.add(tf.keras.layers.Conv2D(128,(3,3), strides = (1), activation='relu'))

#Second maxpooling layer

vgmodel.add(tf.keras.layers.MaxPool2D(pool_size = (3), strides = (2,2)))

#Third convolutional section - consisting of 256 feature map, 3x3 convolutions X2

vgmodel.add(tf.keras.layers.Conv2D(256,(3,3), strides = (1), activation='relu'))

vgmodel.add(tf.keras.layers.Conv2D(256,(3,3), strides = (1), activation='relu'))

#Third maxpooling layer

vgmodel.add(tf.keras.layers.MaxPool2D(pool_size = (3), strides = (2,2)))

#Fourth convolutional section - consisting of 512 feature map, 3x3 convolutions X3

vgmodel.add(tf.keras.layers.Conv2D(512,(3,3), strides = (1), activation='relu'))

vgmodel.add(tf.keras.layers.Conv2D(512,(3,3), strides = (1), activation='relu'))

vgmodel.add(tf.keras.layers.Conv2D(512,(3,3), strides = (1), activation='relu'))

#Third maxpooling layer

vgmodel.add(tf.keras.layers.MaxPool2D(pool_size = (3), strides = (2,2)))

#Fifth convolutional section - consisting of 512 feature map, 3x3 convolutions X3

vgmodel.add(tf.keras.layers.Conv2D(512,(3,3), strides = (1), activation='relu'))

vgmodel.add(tf.keras.layers.Conv2D(512,(3,3), strides = (1), activation='relu'))

vgmodel.add(tf.keras.layers.Conv2D(512,(3,3), strides = (1), activation='relu'))

#Fourth maxpooling layer

vgmodel.add(tf.keras.layers.MaxPool2D(pool_size = (2), strides = (2,2)))

#Flatten model

vgmodel.add(tf.keras.layers.Flatten())

#Add final fully connected layers for classification

vgmodel.add(tf.keras.layers.Dense(4096, activation = "relu"))

vgmodel.add(tf.keras.layers.Dense(4096, activation = "relu"))

#Addd five class softmax finally

vgmodel.add(tf.keras.layers.Dense(5, activation = "softmax"))

#Initialise optimisers to give fine control over parameters

SGDoptimiser = tf.keras.optimizers.SGD(momentum = 0.9, lr=0.01)

RMSPropOptimiser = tf.keras.optimizers.SGD(lr=0.01, momentum=0.9)

#Compile the above model and specify the optimisation function for Stochastic Gradient Decent and categorical crossentropy loss function

vgmodel.compile(optimizer=SGDoptimiser, loss='categorical_crossentropy', metrics=METRICS)

#Print message to indicate model compilation has been completed

print('Model AlexNet compilation complete')

print(vgmodel.summary())

vgmodel.save('/content/drive/My Drive/AD Project/Model Saves/VGG16-Multiclass')

Fit model to dataset

In [ ]:
fittedModel = vgmodel.fit(flowTrainGenerator, epochs=150, steps_per_epoch=17, validation_data=flowValidGenerator, validation_steps=3, verbose=1)

**Visualisations of Performance**

In [ ]:
import matplotlib.pyplot as plt

print(fittedModel.history)

#Plot the accuracy change by number of epochs

plt.plot(fittedModel.history["categorical_accuracy"])
plt.ylabel("Categorical Accuracy")
plt.xlabel("Epoch")
plt.title("Categorical Accuracy by Epoch")
plt.show()

#Plot the change in loss by number of epochs

plt.plot(fittedModel.history["loss"])
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.title("Loss by Epoch")
plt.show()

#Plot the validation accuracy change by number of epochs

plt.plot(fittedModel.history["val_categorical_accuracy"])
plt.ylabel("Validation Accuracy")
plt.xlabel("Epoch")
plt.title("Validation Accuracy by Epoch")
plt.show()

#Plot the change in validaiton loss by number of epochs

plt.plot(fittedModel.history["val_loss"])
plt.ylabel("Validation Loss")
plt.xlabel("Epoch")
plt.title("Validation Loss by Epoch")
plt.show()

**Key Statistics**

Report of key metrics as part of report:

- Accuracy
- F-1 Score
- Precision
- Recall

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, ConfusionMatrixDisplay

val_F1_Score = (2*(((fittedModel.history['val_precision_1'][-1])*(fittedModel.history['val_recall_1'][-1]))/((fittedModel.history['val_precision_1'][-1])+(fittedModel.history['val_recall_1'][-1]))))

F1_Score = (2*(((fittedModel.history['precision_1'][-1])*(fittedModel.history['recall_1'][-1]))/((fittedModel.history['precision_1'][-1])+(fittedModel.history['recall_1'][-1]))))

print("Accuracy is:")

print(fittedModel.history['categorical_accuracy'][-1])

print("F-1 Score is:")

print(F1_Score)

print("Precision is:")

print(fittedModel.history['precision_1'][-1])

print("Recall is:")

print(fittedModel.history['recall_1'][-1])

print("Val Accuracy is:")

print(fittedModel.history['val_categorical_accuracy'][-1])

print("Val F-1 Score is:")

print(val_F1_Score)

print("Val Precision is:")

print(fittedModel.history['val_precision_1'][-1])

print("Val Recall is:")

print(fittedModel.history['val_recall_1'][-1])

print("Confusion Matrix")

classPrediction = fittedModel.predict(flowValidGenerator, verbose=1)

classPrediction = np.argmax(classPrediction, axis=1)

#Use skikit learn maxtrix generator to make confusion matrix with numpy array

confusionMatrix = confusion_matrix(flowValidGenerator.classes,classPrediction)

displayMatrix = ConfusionMatrixDisplay(confusionMatrix, display_labels=flowTrainGenerator.classes)

displayMatrix.plot()